In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

BASE_URL   = "https://www.thegioididong.com/dtdd#c=42&o=13&pi={}"
TIMEOUT    = 15
MAX_PAGES  = 50  # Giới hạn tối đa để tránh infinite loop

options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
driver = webdriver.Chrome(options=options)

def scroll_to_bottom(driver):
    """Scroll từ từ để load hết sản phẩm"""
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Tăng thời gian chờ
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

all_links = []

def detect_last_page(driver):
    """Detect xem có phải trang cuối không"""
    try:
        # Kiểm tra không có sản phẩm nào
        products = driver.find_elements(By.CSS_SELECTOR, "ul.listproduct li")
        if len(products) == 0:
            return True
        return False
    except Exception:
        return False

all_links = []
page = 1
previous_total_count = 0

while page <= MAX_PAGES:
    print(f"Scraping page {page}...")
    
    # Điều hướng đến trang
    driver.get(BASE_URL.format(page))
    
    # Chờ trang load xong
    time.sleep(3)
    
    # REFRESH/RELOAD trang như bấm Ctrl + R
    driver.refresh()
    
    # Chờ sau khi refresh
    time.sleep(3)
    
    try:
        # Scroll để load hết sản phẩm
        scroll_to_bottom(driver)
        
        # Chờ elements xuất hiện
        wait = WebDriverWait(driver, TIMEOUT)
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.listproduct li")))
        
        # Tìm tất cả sản phẩm
        lis = driver.find_elements(By.CSS_SELECTOR, "ul.listproduct li")
        print(f"Found {len(lis)} products on page {page}")
        
        # Nếu không có sản phẩm nào, dừng lại
        if len(lis) == 0:
            print(f"No products found on page {page}. Stopping...")
            break
        
        # Thêm sản phẩm vào danh sách
        valid_products = 0
        for i, li in enumerate(lis, 1):
            try:
                # Thử nhiều selector khác nhau cho link
                a = None
                for selector in ["a", "a[href]", ".product-link", "[href]"]:
                    a_elements = li.find_elements(By.CSS_SELECTOR, selector)
                    if a_elements:
                        a = a_elements[0]
                        break
                
                if not a:
                    continue
                
                # Thử nhiều selector khác nhau cho title
                h3 = None
                title = ""
                for selector in ["h3", ".product-title", ".title", "h3 a", "a"]:
                    h3_elements = li.find_elements(By.CSS_SELECTOR, selector)
                    if h3_elements:
                        h3 = h3_elements[0]
                        title = h3.text.strip()
                        if title:
                            break
                
                if not title:
                    continue
                
                href = a.get_attribute("href")
                
                if href and title:
                    all_links.append({"href": href, "title": title})
                    valid_products += 1
                    
            except Exception as e:
                # Chỉ log lỗi nếu debug mode
                # print(f"Skipping product {i}: {e}")
                continue
        
        print(f"Valid products processed: {valid_products}/{len(lis)}")
        
        # Lọc trùng lặp tạm thời để đếm chính xác
        unique_links_temp = []
        seen = set()
        for link in all_links:
            key = (link["href"], link["title"])
            if key not in seen:
                seen.add(key)
                unique_links_temp.append(link)
        
        current_total_count = len(unique_links_temp)
        print(f"Current total unique products: {current_total_count}")
        
        # KIỂM TRA: Nếu số lượng không tăng lên, dừng lại
        if current_total_count == previous_total_count:
            print(f"No new products found on page {page}. Total count remained at {current_total_count}. Stopping...")
            break
        
        # Cập nhật count cho lần kiểm tra tiếp theo
        previous_total_count = current_total_count
                
    except Exception as e:
        print(f"Error on page {page}: {e}")
        break
    
    # Nghỉ giữa các page để tránh bị block
    time.sleep(2)
    page += 1

print(f"Finished scraping at page {page - 1}")

driver.quit()

# Lọc trùng lặp
unique_links = []
seen = set()

for link in all_links:
    key = (link["href"], link["title"])
    if key not in seen:
        seen.add(key)
        unique_links.append(link)

print(f"Collected {len(unique_links)} unique products")
print("\nFirst 5 products:")
for i, link in enumerate(unique_links[:5], 1):
    print(f"{i}. {link['title']}")
    print(f"   URL: {link['href']}")
    print()

In [30]:
len(unique_links)

131

loop qua mỗi `unique_links`, lấy những thông số cần thiết như màn hình, pin, camera,... 

In [77]:
import requests
from bs4 import BeautifulSoup
import time
import csv

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# ✅ Chuyển hết về lowercase để so khớp
ALLOWED_KEYS = {
    "công nghệ màn hình",
    "độ phân giải màn hình",
    "màn hình rộng",
    "hệ điều hành",
    "độ phân giải camera sau",
    "độ phân giải camera trước",
    "chip xử lý (cpu)",
    "ram",
    "dung lượng lưu trữ",
    "sim",
    "mạng di động",
    "dung lượng pin",
    "hỗ trợ sạc tối đa",
    "hãng"
}

product_details = []

for i, link in enumerate(unique_links, 1):
    url = link["href"]
    title = link["title"]
    print(f"({i}/{len(unique_links)}) Crawling details for: {title}")

    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.encoding = "utf-8"  # ✅ Sửa lỗi encoding
        soup = BeautifulSoup(r.text, "html.parser")

        specs = {}

        # ✅ Loại 1 (chỉ lưu lại các key được phép)
        items = soup.select("div.item.cf-right ul.parameter li")
        if items:
            for li in items:
                span = li.find("span")
                div = li.find("div")
                if span and div:
                    key = span.text.strip().lower().replace(":", "")
                    value = div.text.strip().replace(". Xem thông tin hãng", "")
                    specs[key] = value
                    # if key in ALLOWED_KEYS and value:
                    #     specs[key] = value
        else:
            # ✅ Loại 2 (lọc theo ALLOWED_KEYS)
            boxes = soup.select("div.specification-item div.box-specifi ul li")
            for li in boxes:
                asides = li.find_all("aside")
                if len(asides) >= 2:
                    key = asides[0].text.strip().lower().replace(":", "")
                    value = asides[1].text.strip().replace(". Xem thông tin hãng", "")
                    if key in ALLOWED_KEYS and value:
                        specs[key] = value

        product_details.append({
            "title": title,
            "url": url,
            "specs": specs
        })

        time.sleep(1)  # tránh bị block

    except Exception as e:
        print(f"❌ Error crawling {url}: {e}")
        continue

print(f"\n✅ Crawled details for {len(product_details)} products.")

# ✅ Lưu ra CSV (dùng <br> để ngắt dòng các specs)
with open("products_details.csv", "w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(["title", "url", "specifications"])
    for p in product_details:
        specs_str = "<br>".join([f"{k}: {v}" for k, v in p["specs"].items()])
        writer.writerow([p["title"], p["url"], specs_str])

print("✅ Saved to products_details.csv")


(1/131) Crawling details for: OPPO Reno14 F 5G 12GB/256GB
(2/131) Crawling details for: Samsung Galaxy Z Flip7 FE 5G 8GB/128GB
(3/131) Crawling details for: iPhone 16 Pro Max 256GB
(4/131) Crawling details for: iPhone 16 Pro 128GB
(5/131) Crawling details for: Samsung Galaxy Z Fold7 5G 12GB/256GB
(6/131) Crawling details for: Samsung Galaxy Z Flip7 5G 12GB/256GB
(7/131) Crawling details for: OPPO Reno14 5G 12GB/512GB
(8/131) Crawling details for: OPPO Reno14 Pro 5G 12GB/512GB
(9/131) Crawling details for: Xiaomi Redmi Note 14 Pro 8GB/256GB
(10/131) Crawling details for: Xiaomi Redmi Note 14 Pro+ 5G 8GB/256GB
(11/131) Crawling details for: vivo Y39 5G 8GB/128GB
(12/131) Crawling details for: vivo V50 Lite 5G 8GB/256GB
(13/131) Crawling details for: realme 14T 5G 8GB/256GB
(14/131) Crawling details for: HONOR 400 5G 12GB/256GB Đen
(15/131) Crawling details for: Tecno Spark 30 8GB/128GB
(16/131) Crawling details for: Masstel S9 6GB/256GB
(17/131) Crawling details for: iPhone 16 Plus 128GB